<a href="https://colab.research.google.com/github/Athom77/INVEST_IO/blob/master/Twitter_scraping_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install --user tweepy
import tweepy
import numpy as np
import pandas
from tweepy import OAuthHandler
from tqdm import tqdm_notebook as tqdm 

#First, we have to authorize our "app", we use the keys we generated for this purpose.
#We need the four values to authenticate with OAuth:
#ILA API key:iElgG8dgF2m9cjh1tKWEwNcY9
#ILA API secret key:IAKIHcaRJNUEjhYu3gNxEMV4Ce9NFh4cMaNt7OXM59WLI6QS77
#ILA access_token = '1272418527388073984-FRBZVTP12LOkDR85RLN2WBcJXnM6yQ'
#ILA access_secret = 'GqGN9mOggN4Sjt7DVALiTXHjdaBt1pzF84Vm5Mf1UbhyP'

consumer_key = 'iElgG8dgF2m9cjh1tKWEwNcY9'
consumer_secret = 'IAKIHcaRJNUEjhYu3gNxEMV4Ce9NFh4cMaNt7OXM59WLI6QS77'
access_token = '1272418527388073984-FRBZVTP12LOkDR85RLN2WBcJXnM6yQ'
access_secret = 'GqGN9mOggN4Sjt7DVALiTXHjdaBt1pzF84Vm5Mf1UbhyP'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
query = 'intesa'
searched_tweets = []
last_id = -1
max_tweets = 10000
while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        new_tweets = api.search(q=query, count=100, max_id=str(last_id - 1))
        # If I've reached the end of the search then I'm done
        # immediately the cycle while
        if not new_tweets:
            break
        # added the data found to the list
        searched_tweets.extend(new_tweets)
        # retrieve the last id found
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        print(e)
        break

len(searched_tweets)

9997

In [ ]:
import pandas as pd
# Create our dict with our data
dict_ = {'id': [], 'user': [], 'date': [], 'text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
#
for status_j in searched_tweets:
    status = status_j._json
    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])


df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(10)